In [ ]:
# Installer la bibliothèque Kaggle
!pip install -q kaggle

# Téléverser votre fichier kaggle.json
from google.colab import files
files.upload()
#!mv "kaggle (1).json" kaggle.json

# Configurer Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Télécharger le dataset "Road Sign Detection"
!kaggle datasets download -d andrewmvd/road-sign-detection

# Décompresser le fichier téléchargé
!mkdir road_sign_detection
!unzip road-sign-detection.zip -d road_sign_detection

In [ ]:
!pip install torch torchvision torchaudio transformers pycocotools opencv-python

In [ ]:
!pip install -q ultralytics tensorboard

In [4]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import albumentations as A
import torchaudio
from pycocotools.coco import COCO
import cv2
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, T5TokenizerFast, T5ForConditionalGeneration
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
class_names = ['speedlimit', 'stop', 'crosswalk', 'trafficlight']

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import xml.etree.ElementTree as ET

# Fonction de conversion des annotations de Pascal VOC à YOLO
def convert_voc_to_yolo(voc_dir, yolo_dir, class_names):
    if not os.path.exists(yolo_dir):
        os.makedirs(yolo_dir)

    for xml_file in os.listdir(voc_dir):
        if not xml_file.endswith('.xml'):
            continue

        tree = ET.parse(os.path.join(voc_dir, xml_file))
        root = tree.getroot()

        image_size = root.find('size')
        image_width = int(image_size.find('width').text)
        image_height = int(image_size.find('height').text)

        yolo_annotation = []
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in class_names:
                continue

            class_id = class_names.index(class_name)

            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            x_center = (xmin + xmax) / 2.0 / image_width
            y_center = (ymin + ymax) / 2.0 / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            yolo_annotation.append(f"{class_id} {x_center} {y_center} {width} {height}")

        yolo_file = os.path.join(yolo_dir, xml_file.replace('.xml', '.txt'))
        with open(yolo_file, 'w') as f:
            f.write('\n'.join(yolo_annotation))

# Chemins vers les dossiers
root_dir = '/content/road_sign_detection'
images_dir = os.path.join(root_dir, 'images')
annotations_dir = os.path.join(root_dir, 'annotations')

train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

# Créer les répertoires de sortie
os.makedirs(train_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'annotations'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)

os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'annotations'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

os.makedirs(test_dir, exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'annotations'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)

# Obtenir les chemins des fichiers et trier
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png') or f.endswith('.jpg')])
annotation_files = sorted([f for f in os.listdir(annotations_dir) if f.endswith('.xml')])

# Vérification des fichiers trouvés
print("Images trouvées:", image_files)
print("Annotations trouvées:", annotation_files)

# Diviser les données en train (80%), val (10%) et test (10%)
train_images, test_images, train_annotations, test_annotations = train_test_split(
    image_files, annotation_files, test_size=0.2, random_state=42, shuffle=False
)

val_images, test_images, val_annotations, test_annotations = train_test_split(
    test_images, test_annotations, test_size=0.5, random_state=42, shuffle=False
)

def move_files(file_list, source_dir, dest_dir):
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

# Déplacer les fichiers
move_files(train_images, images_dir, os.path.join(train_dir, 'images'))
move_files(train_annotations, annotations_dir, os.path.join(train_dir, 'annotations'))

move_files(val_images, images_dir, os.path.join(val_dir, 'images'))
move_files(val_annotations, annotations_dir, os.path.join(val_dir, 'annotations'))

move_files(test_images, images_dir, os.path.join(test_dir, 'images'))
move_files(test_annotations, annotations_dir, os.path.join(test_dir, 'annotations'))

# Convertir les annotations pour les ensembles d'entraînement, de validation et de test
convert_voc_to_yolo('/content/road_sign_detection/train/annotations', '/content/road_sign_detection/train/labels', class_names)
convert_voc_to_yolo('/content/road_sign_detection/val/annotations', '/content/road_sign_detection/val/labels', class_names)
convert_voc_to_yolo('/content/road_sign_detection/test/annotations', '/content/road_sign_detection/test/labels', class_names)


In [ ]:
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

# Définir les classes
class_names = ['speedlimit', 'stop', 'crosswalk', 'trafficlight']

# Initialiser le dictionnaire des occurrences
class_counts = {class_name: 0 for class_name in class_names}

# Dossiers des annotations
annotations_dirs = [
    '/content/road_sign_detection/train/annotations',
    '/content/road_sign_detection/val/annotations',
    '/content/road_sign_detection/test/annotations'
]

# Parcourir tous les dossiers d'annotations
for annotations_dir in annotations_dirs:
    # Parcourir tous les fichiers d'annotations dans le dossier
    for filename in os.listdir(annotations_dir):
        if filename.endswith('.xml'):
            file_path = os.path.join(annotations_dir, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Parcourir toutes les balises 'object' pour trouver les étiquettes
            for obj in root.findall('object'):
                label = obj.find('name').text
                if label in class_counts:
                    class_counts[label] += 1

# Afficher le dictionnaire des occurrences
print(class_counts)

# Tracer l'histogramme des occurrences
plt.bar(class_counts.keys(), class_counts.values(), color='blue')
plt.xlabel('Classes')
plt.ylabel('Occurrences')
plt.title('Occurrences des Classes dans les Annotations')
plt.show()


In [ ]:
!pip install gtts

In [ ]:
!pip install gtts playsound opencv-python-headless


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from ultralytics import YOLO
from matplotlib import pyplot as plt
import cv2
from gtts import gTTS
import IPython.display as ipd

# Charger le modèle YOLOv5 pré-entraîné
model = YOLO ('/content/best_model (5).pt')

# Méthode pour obtenir une couleur unique pour chaque classe basée sur l'indice de la classe
def get_color(index):
    colors = [
        (0, 255, 0),   # Vert
        (255, 0, 0),   # Rouge
        (0, 0, 255),   # Bleu
        (255, 255, 0), # Jaune
        (0, 255, 255), # Cyan
        (255, 0, 255)  # Magenta
    ]
    return colors[index % len(colors)]

# Chemin vers le répertoire des images de test
test_images_dir = '/content/road_sign_detection/test/images'
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.png') or img.endswith('.jpg')]

# Générer une description des résultats
def generate_description(results):
    descriptions = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            score = float(box.conf)
            descriptions.append(f"Detected {model.names[class_id]} with confidence {score:.2f}")
    return " ".join(descriptions)

# Convertir le texte en parole
def text_to_speech(text):
    if text:
        tts = gTTS(text=text, lang='en')
        tts.save("output.mp3")
        ipd.display(ipd.Audio("output.mp3"))
    else:
        print("No detections to speak about.")

# Parcourir les images de test et afficher les résultats
for img_path in test_images:
    results = model(img_path)
    description = generate_description(results)
    print(description)
    text_to_speech(description)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for result in results:
        for box, cls, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            class_id = int(cls)
            class_name = model.names[class_id]
            confidence = float(conf) * 100  # Convertir en pourcentage
            label = f"{class_name}: {confidence:.1f}%"
            color = get_color(class_id)  # Obtenir la couleur basée sur l'indice de la classe

            # Dessiner les boîtes de délimitation et les labels
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, color, 1, cv2.LINE_AA)  # Utilisation d'une police plus lisible

    # Augmenter la taille de l'image de sortie
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(description)
    plt.axis("off")
    plt.show()
